In [ ]:
!pip install geopandas
!pip install geopy
!pip install pyproj

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re
from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()
from shapely.geometry import Point as shapely_Point
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime

#reproject import 필요



def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)

In [2]:
import numpy as np
import pandas as pd
import folium
from folium import plugins
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score,accuracy_score, f1_score


from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics import silhouette_score
from itertools import combinations
center = (37.1623799231016, 127.05436890115905)
plt.style.use(['ggplot'])
rs = 123
np.random.seed(rs)
random.seed(rs)

In [3]:
#tab에서 찾기 편하게 모든 단어에 _count_ 붙음
#refer_data 는 data_set['function name']
#홍익선
overspeed_cam_count_ = pd.read_csv('overspeed_cam.csv')
floating_pop_count_ = pd.read_csv('floating_pop.csv')
child_pedestrian_count_ = pd.read_csv('child_pedestrian.csv')
bump_count_ = pd.read_csv('bump.csv')
parking_count_ = pd.read_csv('parking.csv')
parking_cctv_count_ = [parking_count_, pd.read_csv('parking_cctv.csv')]

#조민주
car_count_ = gpd.read_file("3.오산시_차량등록현황_격자.geojson", encoding = "utf-8")
child_count_ = gpd.read_file("child_count.geojson", encoding = "utf-8")
child_rate_count_ = gpd.read_file("child_rate.geojson", encoding = "utf-8")
elem_kinder_count_ = gpd.read_file("elem_kinder_count.csv", encoding = "utf-8")
isSchoolZone_count_ = pd.read_csv("9.오산시_어린이보호구역(re).csv", encoding = "utf-8")
numberSchoolZone_count_ = isSchoolZone_count_

#심재훈
accident_count_ = pd.read_csv("accident_count.csv")
chaos1_nearby_count_ = [gpd.read_file('23.오산시_상세도로망_LV6.geojson'),
                 pd.read_csv('chaos1.csv')]
width_nearby_count_ = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
cross_road_nearby_count_ = width_nearby_count_

#이재훈
num_cram_school_count_ = pd.read_csv("학원.csv")
shortest_cross_count_ = gpd.read_file("횡단보도.json",encoding="utf-8")
shortest_traffic_signal_count_ = gpd.read_file('19.오산시_신호등.geojson',encoding="utf-8")
shortest_sidewalk_count_ =  gpd.read_file("인도.json",encoding="utf-8")


data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'child_pedestrian_count':child_pedestrian_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
}

#only <= 12
data_set['accident_count'].loc[:,'가해운전자 연령'] = data_set['accident_count'].loc[:,'가해운전자 연령'].apply(lambda x: int(x[:-1]) if x[:-1].isnumeric() else 999)
data_set['accident_count'].loc[:,'피해운전자 연령'] = data_set['accident_count'].loc[:,'피해운전자 연령'].apply(lambda x: int(x[:-1]) if str(x)[:-1].isnumeric() else 999)

mask1 = data_set['accident_count'].loc[:,'가해운전자 연령'] <= 12
mask2 = data_set['accident_count'].loc[:,'피해운전자 연령'] <= 12

data_set['accident_count'] = data_set['accident_count'].loc[mask1|mask2]
data_set['accident_count'].index = range(len(data_set['accident_count']))

#tuple이 str으로 저장되는 문제
def _str_to_tuple(string):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  return re.findall(r'\d+\.\d+',string)

def overspeed_cam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)
#from https://medium.com/analytics-vidhya/calculating-distances-from-points-to-polygon-borders-in-python-a-paris-example-3b597e1ea291
def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line
    
#additional interrial function
def _multiline_to_arr_cords(mls):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  arr_cords = []
  lon_lat = re.findall(r'\d+\.\d+',str(mls))
  for i in range(0,len(lon_lat),2):
    arr_cords.append((float(lon_lat[i]),float(lon_lat[i+1])))
  return arr_cords

#(lat, lon), (lon, lat) 자꾸 갈려서 나오네
def _shPoint_to_list(shpoint):
  c1, c2 = re.findall(r'\d+\.\d+',str(shpoint))
  if c1 < c2:
    lon, lat = float(c1), float(c2)
  else:
    lat, lon = float(c1), float(c2)
  return lat, lon

def _list_to_gpPoint(mylist):
  return Point(latitude=mylist[0], longitude=mylist[1])

def _shPoint_to_gpPoint(shpoint):
  mylist = _shPoint_to_list(shpoint)
  return Point(latitude=mylist[0], longitude=mylist[1])

def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(m), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  #watchout for (lat lon), (lon lat)
  output: following roads
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  road_nearby(lat, lon, radi, road_gdf)
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0])
  result = road_gdf.loc[dist_array < radi,:]
  while (result.size == 0):
    radi += 5
    result = road_gdf.loc[dist_array < radi,:]
  return result


In [5]:
#설치 도로 찾기
overspeeds = pd.read_csv("overspeed_cam.csv")
bumps = pd.read_csv("bump.csv")
roads_gdf = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

bump_to_nearest_roads = []
overspeeds_to_nearest_roads = []
for i in range(len(bumps)):
  lat, lon = bumps.loc[:,['lat','lon']].loc[i,:]
  bump_to_nearest_roads.append(road_nearby(lat,lon,1,roads_gdf).link_id.iloc[0])
for i in range(len(overspeeds)):
  lat, lon = overspeeds.loc[:,['lat','lon']].loc[i,:]
  overspeeds_to_nearest_roads.append(road_nearby(lat,lon,1,roads_gdf).link_id.iloc[0])

#선형 회귀
speed = pd.read_csv('speed.csv').iloc[:,1:]
speed = speed.loc[speed.week_type == 'weekend']
speed.loc[:,'bumps'] = speed.apply(lambda x: str(x.link_id) in bump_to_nearest_roads, axis= 1)
speed.loc[:,'cam'] = speed.apply(lambda x: str(x.link_id) in overspeeds_to_nearest_roads, axis= 1)
roads_gdf.link_id = roads_gdf.link_id.apply(int)
final_df = pd.merge(speed,roads_gdf,how='left',left_on='link_id',right_on='link_id')
final_df = final_df.loc[final_df.link_type.apply(lambda x: x in ['32768','2','1','32','64','8'])] #교차로(4)는 noise로 취급

X = pd.concat([final_df.cam.apply(int),final_df.bumps.apply(int),pd.get_dummies(final_df.road_rank),pd.get_dummies(final_df.link_type),pd.get_dummies(final_df.road_type),pd.get_dummies(final_df.max_speed)],axis=1)
Y = final_df.loc[:,['velocity_AVRG']]

X = MinMaxScaler().fit_transform(X)
Y = MinMaxScaler().fit_transform(Y)

model = sm.OLS(Y,sm.add_constant(X))
results = model.fit()
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
===================================================================
Model:              OLS              Adj. R-squared:     0.480     
Dependent Variable: y                AIC:                -1264.7096
Date:               2021-01-26 06:29 BIC:                -1140.8914
No. Observations:   1046             Log-Likelihood:     657.35    
Df Model:           24               F-statistic:        41.17     
Df Residuals:       1021             Prob (F-statistic): 6.30e-132 
R-squared:          0.492            Scale:              0.017068  
---------------------------------------------------------------------
             Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
---------------------------------------------------------------------
const        0.3407     0.0137   24.7770   0.0000    0.3137    0.3677
x1          -0.0232     0.0289   -0.8032   0.4221   -0.0800    0.0335
x2          -0.0620     0.0316   -1.9638   0.0498   -0.1239   -0.0000
x3           0.2887     0.0288   10.0190   0.0000    0.2322    0.3452
x4          -0.0133     0.0194   -0.6857   0.4931   -0.0515    0.0248
x5          -0.0477     0.0166   -2.8626   0.0043   -0.0803   -0.0150
x6           0.0092     0.0169    0.5422   0.5878   -0.0241    0.0424
x7          -0.0196     0.0121   -1.6177   0.1060   -0.0433    0.0042
x8           0.1233     0.0425    2.8996   0.0038    0.0399    0.2068
x9           0.1389     0.0233    5.9737   0.0000    0.0933    0.1845
x10          0.2761     0.0324    8.5136   0.0000    0.2125    0.3398
x11         -0.0663     0.0277   -2.3964   0.0167   -0.1206   -0.0120
x12          0.0100     0.0178    0.5634   0.5733   -0.0249    0.0450
x13         -0.1640     0.0480   -3.4151   0.0007   -0.2582   -0.0698
x14          0.1459     0.0427    3.4148   0.0007    0.0621    0.2297
x15         -0.0186     0.0163   -1.1381   0.2553   -0.0505    0.0134
x16          0.2752     0.0432    6.3629   0.0000    0.1903    0.3600
x17          0.0828     0.0481    1.7203   0.0857   -0.0116    0.1772
x18          0.0241     0.0277    0.8700   0.3845   -0.0302    0.0784
x19          0.0783     0.0358    2.1871   0.0290    0.0081    0.1486
x20         -0.1012     0.0545   -1.8569   0.0636   -0.2081    0.0057
x21         -0.0840     0.0225   -3.7316   0.0002   -0.1281   -0.0398
x22          0.1169     0.0473    2.4727   0.0136    0.0241    0.2096
x23          0.1718     0.0483    3.5568   0.0004    0.0770    0.2666
x24          0.0044     0.0215    0.2026   0.8395   -0.0379    0.0466
x25         -0.0863     0.0206   -4.1861   0.0000   -0.1267   -0.0458
x26         -0.0974     0.0143   -6.8055   0.0000   -0.1255   -0.0693
x27          0.0239     0.0097    2.4474   0.0146    0.0047    0.0430
x28          0.1093     0.0279    3.9204   0.0001    0.0546    0.1639
x29          0.1821     0.0125   14.6208   0.0000    0.1577    0.2066
-------------------------------------------------------------------
Omnibus:           33.658    Durbin-Watson:       0.921            
Prob(Omnibus):     0.000     Jarque-Bera (JB):    41.404           
Skew:              0.361     Prob(JB):            0.000            
Kurtosis:          3.654     Condition No.:       11999392143049248
===================================================================
* The condition number is large (1e+16). This might indicate
strong multicollinearity or other numerical problems.
"""